In [1]:
import pandas as pd
from utils.BertAugmentation import BertAugmentation
from utils.adverb_augmentation import AdverbAugmentation
import transformers

/home/cho/anaconda3/envs/augmen/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BERT_aug = BertAugmentation()
random_masking_replacement = BERT_aug.random_masking_replacement
random_masking_insertion = BERT_aug.random_masking_insertion

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


## BERT augmentation

In [3]:
sentence = "아버지가 오늘 아침에 부엌에 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다."
ratio = 0.15

In [4]:
print('random_masking_replacement:', random_masking_replacement(sentence, ratio))
print('random_insertion_replacement:', random_masking_insertion(sentence, ratio))

random_masking_replacement: 아버지가 잠시 부엌에 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다.
random_insertion_replacement: 아버지가 오늘 아침에는 부엌에 와 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다.


In [5]:
train_df = pd.read_csv('/home/cho/test/aiffel-dl-thon-dktc-online-12/train.csv')
train_df.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [6]:
normal_df = pd.read_csv('/home/cho/test/aiffel-dl-thon-dktc-online-12/일반대화 합성데이터(GPT-4o, AIhub 참고).csv')
normal_df.head()

,idx,class,conversation
0,0,일반 대화,이번에 캘리포니아에 산불 난 거 보셨어요?\n네 봤어요 ㅜㅜ 일주일 넘게 진압하고 ...
1,1,일반 대화,얼마 전에 동네 피부 미용샵에 다녀왔어요! 여기 좋더라고요!\n동네에 그런 곳이 있...
2,2,일반 대화,코로나 터지고 건강에 더 주의를 기울이게 된 것 같아요\n인정이요. 위생에도 더 철...
3,3,일반 대화,"요즘 택배가 참 편하긴 한 것 같아요\n맞아요, 옛날엔 몰랐는데 새벽 배송에 로켓 ..."
4,4,일반 대화,다들 TV프로그램 뭐 보세요?\n요즘 볼게 없어요\n전 슬기로운 의사생활 재밌게 봤...


In [7]:
# train data 와 normal 데이터 합치기
def concat_train_normal(train_df, normal_df):
    train_df = pd.concat([train_df, normal_df], ignore_index=True)
    train_df.drop(['idx'], axis=1, inplace=True)
    train_df.reset_index(inplace=True)

    return train_df

train_df = concat_train_normal(train_df, normal_df)
train_df.head()

,index,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [8]:
# 클래스 숫자로 변환
def class_encoding(train_df, class_mapping):
    train_df['class_encoded'] = train_df['class'].replace(class_mapping)
    
    return train_df

# 클래스 매핑 딕셔너리 정의
class_mapping = {
    "협박 대화": 0,
    "갈취 대화": 1,
    "직장 내 괴롭힘 대화": 2,
    "기타 괴롭힘 대화": 3,
    "일반 대화": 4
}

train_df = class_encoding(train_df, class_mapping)

# 변환된 데이터 확인
train_df.head()


,index,class,conversation,class_encoded
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,0
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,0
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,3
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,1
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,1


In [9]:
# 중복된 'conversation' 제거
train_df = train_df.drop_duplicates(subset=['conversation'])

# 중복 제거 후 데이터 확인
print(f"중복 제거된 후 데이터 수: {train_df.shape[0]}")

중복 제거된 후 데이터 수: 4846


In [10]:
train_df = train_df.reset_index(drop=True)

In [11]:
train_df.to_csv('raw_data.csv',index=False)

In [12]:
CLASS = train_df.loc[392,'class']
class_encoded = train_df.loc[392,'class_encoded']
conversation = train_df.loc[392,'conversation']

replaced_text = random_masking_replacement(conversation, ratio)
inserted_text = random_masking_insertion(conversation, ratio)

new_data_replaced = {'class' : CLASS, 
            'class_encoded' : class_encoded, 
            'conversation' : replaced_text
           }

new_data_inserted = {'class' : CLASS, 
            'class_encoded' : class_encoded, 
            'conversation' : inserted_text
           }

print(new_data_replaced)

{'class': '직장 내 괴롭힘 대화', 'class_encoded': 2, 'conversation': '회식 갈거지? 오늘 몸이 좀 안 아파 오늘 왜 잘 못해. 한다고 하는데 뜻대로 안되네요. 오늘 회식은 과장님 축하 위한거니 되도록이면 모두 참석 했으면 하는데. 죄송합니다. 그 정도로 될 것 같지 않은데 잠깐이라도 참석해서 더 이상 안나오게 해. 잘 알겠습니다. 사람이 참 융통성이 없어. 죄송합니다.'}


In [ ]:
import pandas as pd

# 결과 저장을 위한 리스트
replaced_text_data = []
inserted_text_data = []

# 비율 설정
ratio = 0.15

# 모든 행에 대해 데이터 변환 수행
for i in range(len(train_df)):
    try:
        cls = train_df.loc[i, 'class']
        class_encoded = train_df.loc[i, 'class_encoded']
        conversation = train_df.loc[i, 'conversation']
        
        replaced_text = random_masking_replacement(conversation, ratio)
        inserted_text = random_masking_insertion(conversation, ratio)
        
        # 변형된 데이터 추가
        replaced_text_data.append({'class': cls, 'class_encoded': class_encoded, 'conversation': replaced_text})
        inserted_text_data.append({'class': cls, 'class_encoded': class_encoded, 'conversation': inserted_text})
    
    except Exception as e:
        print(f"Error at index {i}: {e}")
        continue  # 오류 발생 시 해당 인덱스를 건너뛰고 다음으로 진행

# 변형된 데이터를 DataFrame으로 변환
augmented_replaced_text_data = pd.DataFrame(replaced_text_data)
augmented_inserted_text_data = pd.DataFrame(inserted_text_data)

augmented_replaced_text_data.head()

## Adverb augmentation

In [ ]:
from quickspacer import Spacer
spacer = Spacer()
spacer.space(['생각할겨를도없이빨리'])

In [5]:
sentence = "아버지가 부엌에 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다."

In [ ]:
adverb_gloss_replacement(sentence)

## 기존 EDA 방법 (KorEDA)
- random_delete
- random_swap
- synonym_replacement
- random_insert

In [8]:
from utils.koreda import synonym_replacement, random_deletion, random_swap, random_insertion

In [9]:
sentence = "이순신은 조선 중기의 매우 뛰어난 전략가이자 용맹한 무신이다."

In [ ]:
print('synonym_replacement: ', " ".join(synonym_replacement(sentence.split(), 1)))
print('random_deletion: ', " ".join(random_deletion(sentence.split(), 0.15)))
print('random_swap: ', " ".join(random_swap(sentence.split(), 1)))
print('random_insertion: ', " ".join(random_insertion(sentence.split(), 1)))

## AEDA

In [11]:
from utils.aeda import aeda

In [12]:
sentence = "이순신은 조선 중기의 매우 뛰어난 전략가이자 용맹한 무신이다."

In [ ]:
print('aeda: ', aeda(sentence))

# Gemini 1.5 flash

In [1]:
from utils.LLM_augmentation import LlmAugmentation

/home/bs/anaconda3/envs/ktacc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm_aug = LlmAugmentation()

In [3]:
sentences = \
[
    "이순신은 조선 중기의 매우 뛰어난 전략가이자 용맹한 무신이다.",
    "하늘을 우러러 한 점 부끄럼 없기를",
    "꽁꽁 얼어붙은 한강 위로 고양이가 걸어다닙니다",
    "암세포도 생명이야...!",
    "겨울은 강철로 된 무지개인가 보다.",
    "나랑 별 보러 가지 않을래?",
    "피할 수 없다면 즐겨라.",
    "인생은 멀리서 보면 희극, 가까이서 보면 비극",
    "그래도 지구는 돈다.",
    "하루하루는 열심히 인생은 대충대충",
    "고딩 친구 = 평생 친구",
    "롤 한판 하쉴?",
    "365에 13을 더하면?",
    "1995년 12월 23일은 철수의 생일이다."
]

paraphrased_sentences = llm_aug.generate_paraphrased_sentence(sentences)

In [4]:
paraphrased_sentences

['이순신 장군은 조선 중기 최고의 전략가이자 용맹한 무장으로 손꼽힌다.',
 '하늘에 부끄러울 일이 없도록 살자.',
 '꽁꽁 언 한강 위를 고양이가 걸어 다니는 진풍경이 펼쳐진다.',
 '암세포도 생명의 한 형태이다.',
 '겨울은 마치 강철로 만든 무지개처럼 차가운 아름다움을 지닌다.',
 '함께 밤하늘의 별을 보러 가지 않겠어?',
 '피할 수 없다면 즐겁게 받아들이자.',
 '인생은 멀리서 보면 희극처럼 보이지만, 가까이서 보면 비극처럼 느껴진다.',
 '그래도 지구는 끊임없이 자전하고 있다.',
 '하루하루는 최선을 다하고, 인생은 좀 느긋하게 살자.',
 '고등학교 친구는 평생을 함께하는 소중한 인연이다.',
 '롤 한 게임 같이 할래?',
 '365에 13을 더하면 378이 됩니다.',
 '철수의 생일은 1995년 12월 23일이다.']